# OpenAI Agents SDK - Calendar Research Assistant

Welcome to AI Makerspace! This notebook demonstrates the powerful new **OpenAI Agents SDK**, which provides a framework for building intelligent agents that can interact with external tools and services.

## What We'll Build

A **Calendar Research Assistant** that:
1. **Connects to your Google Calendar** to fetch events for a specific day
2. **Analyzes each event** to understand what preparation might be needed
3. **Performs web research** to gather relevant information about event topics
4. **Creates structured preparation guides** with key insights and recommendations

## Key Features of the Agents SDK

- **Tool Integration** - Connect to external services like calendars and web search
- **Structured Reasoning** - Built-in reasoning controls for complex decision-making
- **Async Support** - Efficient parallel processing of multiple tasks
- **Streaming Responses** - Real-time updates as the agent works
- **Type Safety** - Pydantic models for structured inputs and outputs
- **MCP Connectors** - Model Context Protocol support for service integration

This notebook walks through building a practical agent that helps you prepare for meetings and events by automatically researching relevant topics and providing actionable insights.


### Setup and Authentication

First, we need to set up our authentication. We'll need:
1. **OpenAI API Key** - For the agents and language models
2. **Google Calendar Authorization** - To access your calendar events

For Google Calendar:
1. Visit [Google OAuth Playground](https://developers.google.com/oauthplayground/)
2. Input `https://www.googleapis.com/auth/calendar.events` as the required scope
3. Click "Authorize APIs" and go through the OAuth flow
4. Click "Exchange authorization code for tokens"
5. Copy the access token (starts with "ya29.")

> **Note**: The access token expires after about an hour. For production use, implement proper OAuth refresh token handling.


In [23]:
import os
import getpass

# Set up OpenAI API Key
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

# Set up Google Calendar Authorization
print("\nFollow the instructions above to get your Google Calendar access token.")
os.environ["GOOGLE_CALENDAR_AUTHORIZATION"] = getpass.getpass("Google Calendar Access Token (ya29...): ")



Follow the instructions above to get your Google Calendar access token.


### Import Required Libraries

Now we'll import the necessary components from the OpenAI Agents SDK and other required libraries. The SDK provides:

- `Agent` - The core agent class that defines behavior and capabilities
- `HostedMCPTool` - Model Context Protocol tool for connecting to external services
- `WebSearchTool` - Built-in tool for web search capabilities
- `Runner` - Executes agents with their inputs
- `ModelSettings` - Configuration for model behavior and reasoning


In [25]:
import asyncio
import json
from datetime import datetime, timedelta
from typing import List, Optional
from pydantic import BaseModel

from agents import Agent, HostedMCPTool, WebSearchTool, Runner, ModelSettings
from openai.types.shared.reasoning import Reasoning


### Define Structured Output Models

We'll use Pydantic models to define structured outputs for our agents. This ensures type safety and provides clear contracts for the data our agents will produce.

These models define:
- **CalendarEvent** - Structure for calendar event data
- **EventResearch** - Research findings about a specific event
- **PreparationGuide** - Actionable preparation recommendations


In [26]:
class CalendarEvent(BaseModel):
    """Represents a calendar event with essential details."""
    title: str
    start_time: str
    end_time: str
    location: Optional[str]
    attendees: List[str]
    description: Optional[str]
    
class ResearchTopic(BaseModel):
    """A specific topic to research related to an event."""
    topic: str
    relevance: str
    search_query: str

class EventResearch(BaseModel):
    """Research findings for a calendar event."""
    event_title: str
    event_type: str  # meeting, presentation, interview, etc.
    key_topics: List[str]
    research_queries: List[ResearchTopic]
    
class PreparationItem(BaseModel):
    """A specific preparation action item."""
    action: str
    priority: str  # high, medium, low
    time_estimate: str
    resources: List[str]

class PreparationGuide(BaseModel):
    """Complete preparation guide for an event."""
    event_title: str
    event_time: str
    summary: str
    key_insights: List[str]
    preparation_checklist: List[PreparationItem]
    talking_points: List[str]
    questions_to_ask: List[str]
    potential_challenges: List[str]
    recommended_reading: List[str]


### Create the Calendar Agent

Our first agent connects to Google Calendar using the MCP (Model Context Protocol) connector. This agent:

- Uses `HostedMCPTool` to integrate with Google Calendar
- Fetches events for a specific date
- Returns structured event information
- Handles authorization through the OAuth token we provided

The MCP connector provides a standardized way to interact with external services, handling the complexity of API calls behind the scenes.


In [ ]:
calendar_agent = Agent(
    name="CalendarAssistant",
    instructions="""You are a calendar assistant that helps users fetch and understand their schedule.
    When asked about events for a specific day, retrieve all calendar events for that day and provide
    a clear, structured summary of the schedule. Include event titles, times, locations, and attendees.""",
    tools=[
        HostedMCPTool(
            tool_config={
                "type": "mcp",
                "server_label": "google_calendar",
                "connector_id": "connector_googlecalendar",
                "authorization": os.environ["GOOGLE_CALENDAR_AUTHORIZATION"],
                "require_approval": "never",
            }
        )
    ],
    model="gpt-5",
)

### Create the Event Analyzer Agent

The Event Analyzer examines calendar events and determines what research topics would be most helpful for preparation. It:

- Analyzes event titles, descriptions, and attendee lists
- Identifies the type of event (meeting, presentation, interview, etc.)
- Generates relevant research queries
- Uses structured output for consistent results
- Applies medium reasoning effort for thoughtful analysis


In [28]:
event_analyzer_agent = Agent(
    name="EventAnalyzer",
    instructions="""You are an expert at analyzing calendar events to determine what preparation would be helpful.
    Given a calendar event, analyze its title, description, attendees, and context to:
    1. Identify the type of event (meeting, presentation, interview, workshop, etc.)
    2. Extract key topics that should be researched
    3. Generate 3-5 specific, actionable research queries that would help someone prepare
    
    Focus on practical, relevant research that would actually help in the meeting.
    For example:
    - For a product meeting: latest features, competitor analysis, user feedback
    - For a technical discussion: relevant technologies, best practices, recent developments
    - For a client meeting: company background, recent news, industry trends
    
    Make search queries specific and likely to return useful results.""",
    model="gpt-5",
    model_settings=ModelSettings(reasoning=Reasoning(effort="medium")),
    output_type=EventResearch,
)


### Create the Research Agent

The Research Agent performs web searches to gather information about event topics. This agent:

- Uses the built-in `WebSearchTool` for real-time web searches
- Processes multiple search queries in parallel for efficiency
- Summarizes findings into digestible insights
- Focuses on recent, relevant information
- Uses `tool_choice="required"` to ensure it always performs searches


In [29]:
research_agent = Agent(
    name="ResearchAgent",
    instructions="""You are a research assistant specializing in gathering relevant information for meeting preparation.
    Given a search query, perform a comprehensive web search and provide a concise summary of the findings.
    
    Focus on:
    - Recent information (prioritize content from the last 6 months)
    - Authoritative sources
    - Practical insights that would be useful in a meeting context
    - Key facts, trends, and developments
    
    Your summary should be 2-3 paragraphs, highlighting the most important and actionable information.
    Always cite sources when possible and note any particularly interesting or surprising findings.""",
    model="gpt-4.1",
    tools=[WebSearchTool()],
    model_settings=ModelSettings(tool_choice="required"),
)


### Create the Preparation Guide Agent

The Preparation Guide Agent synthesizes all research into an actionable preparation guide. Features:

- Combines event details with research findings
- Generates structured preparation checklists
- Creates talking points and questions to ask
- Identifies potential challenges
- Provides recommended reading
- Uses high reasoning effort for comprehensive analysis


In [30]:
preparation_guide_agent = Agent(
    name="PreparationGuideAgent",
    instructions="""You are an expert meeting preparation coach who creates comprehensive, actionable preparation guides.
    Given event details and research findings, create a detailed preparation guide that helps someone be fully prepared.
    
    Your guide should be:
    - Practical and actionable with specific steps
    - Prioritized by importance and urgency
    - Time-conscious (realistic time estimates)
    - Comprehensive but not overwhelming
    
    Include:
    - A concise summary of what the meeting is about
    - Key insights from the research
    - Specific preparation actions with time estimates
    - Smart talking points based on the research
    - Thoughtful questions to ask during the meeting
    - Potential challenges or sensitive topics to be aware of
    - Links or resources for deeper reading
    
    Tailor the guide to the specific type of event (presentation, interview, client meeting, etc.)""",
    model="gpt-5",
    model_settings=ModelSettings(reasoning=Reasoning(effort="high")),
    output_type=PreparationGuide,
)


### Build the Calendar Research Orchestrator

Now we'll create the main orchestrator function that coordinates all our agents. This function:

1. **Fetches calendar events** for the specified date
2. **Analyzes each event** to determine research needs  
3. **Performs parallel research** on all identified topics
4. **Generates preparation guides** for each event
5. **Returns comprehensive results** with all preparation materials

The orchestrator uses `asyncio` for efficient parallel processing, allowing multiple research queries to run simultaneously.


In [31]:
class CalendarResearchOrchestrator:
    """Orchestrates the calendar research workflow."""
    
    def __init__(self):
        self.calendar_agent = calendar_agent
        self.analyzer_agent = event_analyzer_agent
        self.research_agent = research_agent
        self.guide_agent = preparation_guide_agent
        
    async def research_event(self, event_info: str) -> dict:
        """Research a single calendar event."""
        print(f"\n📅 Analyzing event...")
        
        # Analyze the event to determine research topics
        analysis_result = await Runner.run(
            self.analyzer_agent,
            f"Analyze this calendar event and determine what research would be helpful:\n\n{event_info}"
        )
        event_research = analysis_result.final_output_as(EventResearch)
        
        # Perform research on all topics in parallel
        print(f"🔍 Researching {len(event_research.research_queries)} topics...")
        research_tasks = []
        for topic in event_research.research_queries:
            task = asyncio.create_task(self.perform_research(topic.search_query))
            research_tasks.append(task)
        
        research_results = await asyncio.gather(*research_tasks)
        
        # Combine all research into a preparation guide
        print("📝 Creating preparation guide...")
        research_summary = "\n\n".join([r for r in research_results if r])
        
        guide_input = f"""
        Event Details:
        {event_info}
        
        Event Analysis:
        Type: {event_research.event_type}
        Key Topics: {', '.join(event_research.key_topics)}
        
        Research Findings:
        {research_summary}
        """
        
        guide_result = await Runner.run(
            self.guide_agent,
            guide_input
        )
        
        return {
            "event": event_info,
            "analysis": event_research,
            "research": research_results,
            "guide": guide_result.final_output_as(PreparationGuide)
        }
    
    async def perform_research(self, query: str) -> str:
        """Perform a single research query."""
        try:
            result = await Runner.run(
                self.research_agent,
                f"Research this topic: {query}"
            )
            return str(result.final_output)
        except Exception as e:
            print(f"⚠️ Research failed for '{query}': {e}")
            return None
    
    async def prepare_for_date(self, date: str) -> List[dict]:
        """Prepare for all events on a specific date."""
        print(f"📆 Fetching calendar events for {date}...")
        
        # Fetch calendar events
        calendar_result = await Runner.run(
            self.calendar_agent,
            f"Get all my calendar events for {date}. Include event titles, times, locations, attendees, and descriptions."
        )
        
        events_text = str(calendar_result.final_output)
        
        # Parse events (simple parsing - in production, you'd want more robust parsing)
        if "no events" in events_text.lower() or "empty" in events_text.lower():
            print("No events found for this date.")
            return []
        
        # Process each event
        # For simplicity, we'll process the entire calendar output as one block
        # In production, you'd parse individual events
        results = []
        result = await self.research_event(events_text)
        results.append(result)
        
        return results


### Example Usage - Research Today's Events

Let's use our Calendar Research Assistant to prepare for today's events. The orchestrator will:

1. Connect to your Google Calendar
2. Fetch all events for today
3. Analyze each event and determine research needs
4. Perform web searches on relevant topics
5. Generate a comprehensive preparation guide

> **Note**: This example uses real calendar data and performs actual web searches, so results will vary based on your calendar and current information available online.


In [33]:
# Create the orchestrator
orchestrator = CalendarResearchOrchestrator()

# Get today's date
today = datetime.now().strftime("%Y-%m-%d")
print(f"🗓️ Preparing for events on: {today}")

# Run the preparation workflow
async def run_preparation():
    results = await orchestrator.prepare_for_date(today)
    
    if not results:
        print("\n✨ No events to prepare for today!")
        return
    
    # Display results for each event
    for i, result in enumerate(results, 1):
        print(f"\n{'='*60}")
        print(f"EVENT {i} PREPARATION GUIDE")
        print('='*60)
        
        guide = result['guide']
        
        print(f"\n📌 Event: {guide.event_title}")
        print(f"⏰ Time: {guide.event_time}")
        print(f"\n📋 Summary:\n{guide.summary}")
        
        print("\n🔑 Key Insights:")
        for insight in guide.key_insights[:3]:  # Show top 3 insights
            print(f"  • {insight}")
        
        print("\n✅ Top Preparation Actions:")
        for item in guide.preparation_checklist[:3]:  # Show top 3 actions
            print(f"  [{item.priority.upper()}] {item.action} ({item.time_estimate})")
        
        print("\n💬 Suggested Talking Points:")
        for point in guide.talking_points[:3]:  # Show top 3 talking points
            print(f"  • {point}")
        
        print("\n❓ Questions to Ask:")
        for question in guide.questions_to_ask[:3]:  # Show top 3 questions
            print(f"  • {question}")

# Run the async function
await run_preparation()


🗓️ Preparing for events on: 2025-09-16
📆 Fetching calendar events for 2025-09-16...

📅 Analyzing event...
🔍 Researching 5 topics...
📝 Creating preparation guide...

EVENT 1 PREPARATION GUIDE

📌 Event: Talk on OpenAI Agents SDK
⏰ Time: 2025-09-16 8:00 PM – 9:00 PM America/Los_Angeles

📋 Summary:
A 60-minute developer-focused presentation that introduces OpenAI’s Agents SDK and the Responses API, with a practical walkthrough of single- and multi-agent patterns, tools/actions integration, state and memory, observability and guardrails, deployment and rate limits, security and data handling, plus positioning vs LangChain, AutoGen, and Semantic Kernel. Includes short Python and JavaScript demos with clear migration guidance from Assistants API.

🔑 Key Insights:
  • OpenAI introduced the Responses API (Mar 2025) to unify chat-style generation with tool use; the Assistants API is planned for deprecation by mid-2026 per OpenAI’s announcement, with migration guidance forthcoming.
  • Open-sourc

### Example: Research Events for a Specific Date

You can also prepare for events on any specific date. This is useful for:
- Preparing for tomorrow's meetings
- Getting ready for important events later in the week
- Reviewing past events for follow-ups

Simply provide the date in YYYY-MM-DD format:


In [34]:
# Prepare for tomorrow's events
tomorrow = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")
print(f"🗓️ Preparing for tomorrow's events: {tomorrow}")

async def prepare_tomorrow():
    results = await orchestrator.prepare_for_date(tomorrow)
    
    if not results:
        print("\n✨ No events to prepare for tomorrow!")
        return
    
    print(f"\n📊 Found {len(results)} event(s) to prepare for tomorrow")
    
    # Show a brief summary for each event
    for result in results:
        guide = result['guide']
        analysis = result['analysis']
        
        print(f"\n📅 {guide.event_title}")
        print(f"   Type: {analysis.event_type}")
        print(f"   Research topics: {len(analysis.research_queries)}")
        print(f"   Preparation items: {len(guide.preparation_checklist)}")
        print(f"   Time needed: ~{sum([15 if 'high' in item.priority.lower() else 10 if 'medium' in item.priority.lower() else 5 for item in guide.preparation_checklist])} minutes")

# Run the preparation for tomorrow
await prepare_tomorrow()


🗓️ Preparing for tomorrow's events: 2025-09-17
📆 Fetching calendar events for 2025-09-17...

📅 Analyzing event...
🔍 Researching 5 topics...
📝 Creating preparation guide...

📊 Found 1 event(s) to prepare for tomorrow

📅 OpenAI Agents SDK Event
   Type: Product webinar / launch event
   Research topics: 5
   Preparation items: 11
   Time needed: ~105 minutes


### Advanced Configuration: Custom Research Focus

You can customize the research behavior for specific types of events or domains. This example shows how to create a specialized agent for technical meetings:


In [37]:
# Create a specialized technical meeting preparation agent
technical_prep_agent = Agent(
    name="TechnicalMeetingPrep",
    instructions="""You are a technical meeting preparation specialist focusing on engineering discussions.
    
    For technical meetings, emphasize:
    - Latest technology trends and updates
    - Code examples and implementation details
    - Performance benchmarks and comparisons
    - Security considerations
    - Best practices and architectural patterns
    - Open source alternatives
    - Recent blog posts and documentation updates
    
    Generate highly technical, detailed preparation materials suitable for engineering discussions.""",
    model="gpt-5",
    model_settings=ModelSettings(
        reasoning=Reasoning(effort="high"),
    ),
    tools=[WebSearchTool()],
)

# Example: Analyze a technical meeting
async def prepare_technical_meeting():
    event_description = """
    Meeting: AI Engineering Architecture Review
    Time: 2:00 PM - 3:00 PM
    Attendees: CTO, Lead Engineers, ML Team
    Agenda: Discuss RAG architecture, vector databases, and LLM deployment strategies
    """
    
    print("🔧 Preparing for technical meeting...\n")
    
    # Run the technical preparation
    result = await Runner.run(
        technical_prep_agent,
        f"""Prepare comprehensive technical research for this meeting:
        {event_description}
        
        Focus on:
        1. Latest RAG architecture patterns and best practices
        2. Vector database comparison (Pinecone vs Weaviate vs Chroma)
        3. LLM deployment options (self-hosted vs API)
        4. Cost optimization strategies
        5. Recent developments in the field
        """
    )
    
    print("📊 Technical Preparation Summary:")
    print(result.final_output)

# Run the technical preparation
await prepare_technical_meeting()

🔧 Preparing for technical meeting...

📊 Technical Preparation Summary:
Below is a compact, engineering-first prep pack you can drop into the meeting. It emphasizes current best practices, concrete design options, code you can paste, performance/cost levers, and what’s changed recently (as of September 17, 2025, US).

Meeting context
- Meeting: AI Engineering Architecture Review
- Time: 2:00–3:00 PM
- Attendees: CTO, Lead Engineers, ML Team
- Agenda: RAG architecture, vector databases, LLM deployment strategies

Executive summary (recommended stance)
- RAG: Adopt a two-track retrieval strategy: (1) baseline hybrid RAG (dense + keyword) with multi-stage reranking for QA/chat and (2) graph-enabled RAG for discovery/analytics at corpus scale (GraphRAG family), selecting “LazyGraphRAG” when you need global reasoning but can’t afford heavy pre-index summarization. ([docs.weaviate.io](https://docs.weaviate.io/weaviate/concepts/search/hybrid-search?utm_source=openai))
- Vector DB: For managed,

## Summary and Key Takeaways

Congratulations! You've built a sophisticated Calendar Research Assistant using the OpenAI Agents SDK. Here's what we've accomplished:

### 🎯 What We Built

- **Calendar Integration** - Connected to Google Calendar via MCP connectors
- **Multi-Agent System** - Orchestrated 4 specialized agents working together
- **Parallel Processing** - Efficient research using async/await patterns
- **Structured Outputs** - Type-safe results using Pydantic models
- **Streaming Support** - Real-time updates for better user experience
- **Customization** - Specialized agents for different meeting types

### 🔧 Key Components

1. **Calendar Agent** - Fetches events from Google Calendar
2. **Event Analyzer** - Determines research needs for each event
3. **Research Agent** - Performs web searches on relevant topics
4. **Preparation Guide Agent** - Synthesizes findings into actionable guides
5. **Orchestrator** - Coordinates the entire workflow

### 💡 Best Practices Demonstrated

- **Separation of Concerns** - Each agent has a specific, focused role
- **Error Handling** - Graceful fallbacks when research fails
- **Performance Optimization** - Parallel research queries
- **User Experience** - Clear progress indicators and streaming
- **Flexibility** - Easy to extend with new agents or tools

### 🚀 Next Steps

To extend this system, consider:

1. **Add more connectors** - Slack, Email, Notion for richer context
2. **Implement caching** - Store research results to avoid redundant searches
3. **Add personalization** - Learn from user feedback and preferences
4. **Create a UI** - Build a web interface using the streaming capabilities
5. **Schedule automation** - Run preparation automatically each morning
6. **Export options** - Generate PDFs or send summaries via email

### 📚 Resources

- [OpenAI Agents SDK Documentation](https://platform.openai.com/docs/agents)
- [Model Context Protocol (MCP)](https://platform.openai.com/docs/guides/tools-connectors-mcp)
- [Google Calendar API](https://developers.google.com/calendar)
- [Pydantic Documentation](https://docs.pydantic.dev/)
